In [1]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [2]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [3]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwareComponent"])

mc_openapi.doml_mc.intermediate_model.metamodel.DOMLClass(
    name='application_SoftwareComponent',
    superclass='application_ApplicationComponent',
    attributes={
        'isPersistent': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='isPersistent',
            type='Boolean',
            multiplicity=('1', '1'),
            default=False
        ),
        'licenseCost': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='licenseCost',
            type='String',
            multiplicity=('0', '1'),
            default=None
        ),
        'configFile': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='configFile',
            type='String',
            multiplicity=('0', '1'),
            default=None
        )
    },
    associations={
        'exposedInterfaces': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAssociation(
            name='exposedInterfaces',
            class_

### Load the DOML document here:

In [4]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
doml_document_path = "../../tests/doml/nginx-openstack_v2.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2_wrong.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im = parse_doml_model(doc, mm)
print(im)

{'source_code': DOMLElement(name='source_code', class_='commons_SProperty', attributes={'commons_Property::key': 'source_code', 'commons_SProperty::value': '/usr/share/nginx/html/index.html'}, associations={}), 'nginx': DOMLElement(name='nginx', class_='application_SoftwareComponent', attributes={'application_SoftwareComponent::isPersistent': False, 'commons_DOMLElement::name': 'nginx'}, associations={'commons_DOMLElement::annotations': {'source_code'}}), 'app': DOMLElement(name='app', class_='application_ApplicationLayer', attributes={'commons_DOMLElement::name': 'app'}, associations={'application_ApplicationLayer::components': {'nginx'}}), 'ssh_key': DOMLElement(name='ssh_key', class_='infrastructure_KeyPair', attributes={'infrastructure_KeyPair::algorithm': 'RSA', 'infrastructure_KeyPair::keyfile': '/home/user1/.ssh/openstack.key', 'infrastructure_KeyPair::bits': 4096, 'infrastructure_KeyPair::user': 'ubuntu', 'commons_DOMLElement::name': 'ssh_key'}, associations={}), 'out_all': DOM

Attribute addressRanges of multiplicity > 1 not supported yet.
Attribute addressRanges of multiplicity > 1 not supported yet.
Attribute addressRanges of multiplicity > 1 not supported yet.
Attribute addressRanges of multiplicity > 1 not supported yet.


In [5]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_xmi_model
doml_xmi = parse_xmi_model(doc)

In [6]:
print(dir(doml_xmi.eClass))
print(doml_xmi.infrastructure.eClass.ePackage.name)
print(doml_xmi.infrastructure.eClass.eAllReferences().pop().lower)
print(doml_xmi.activeInfrastructure.eClass)

['abstract', 'allInstances', 'defaultValue', 'eAllAttributes', 'eAllOperations', 'eAllReferences', 'eAllStructuralFeatures', 'eAllSuperTypes', 'eAnnotations', 'eAttributes', 'eGenericSuperTypes', 'eOperations', 'ePackage', 'eReferences', 'eStructuralFeatures', 'eSuperTypes', 'eTypeParameters', 'eURIFragment', 'findEOperation', 'findEStructuralFeature', 'getEAnnotation', 'instanceClass', 'instanceClassName', 'instanceTypeName', 'interface', 'name', 'notifyChanged']
infrastructure
0
<EClass name="ConcreteInfrastructure">


In [7]:
from mc_openapi.doml_mc.intermediate_model.doml_element import reciprocate_inverse_associations
reciprocate_inverse_associations(im, inv_assoc)

In [8]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im)

{
    'source_code': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='source_code',
        class_='commons_SProperty',
        attributes={
            'commons_Property::key': 'source_code',
            'commons_SProperty::value': '/usr/share/nginx/html/index.html'
        },
        associations={}
    ),
    'nginx': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='nginx',
        class_='application_SoftwareComponent',
        attributes={
            'application_SoftwareComponent::isPersistent': False,
            'commons_DOMLElement::name': 'nginx'
        },
        associations={'commons_DOMLElement::annotations': {'source_code'}}
    ),
    'app': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='app',
        class_='application_ApplicationLayer',
        attributes={'commons_DOMLElement::name': 'app'},
        associations={'application_ApplicationLayer::components': {'nginx'}}
    ),
  

In [9]:
unbound_elems_n = 0
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [10]:
from z3 import Solver

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel_and_assert_constraints,
    def_attribute_rel_and_assert_constraints,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel_and_assert_constraints(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel_and_assert_constraints(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort,
    inv_assoc
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [11]:
# solver.push()

### Metamodel statistics

In [12]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



Number of classes: 48
Number of tuples in the subclass relation: 125
Number of attributes: 46
Number of attribute assertions: 138
Number of associations: 54
Number of associations assertions: 168


### DOML model statistics

In [13]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

Number of elements: 24
Number of attributes (IM): 58
Number of associations (IM): 35
Number of string symbols: 40
Number of IM attribute assertions: 24
Number of IM association assertions: 576


In [14]:
solver.check()

sat

In [15]:
solver.unsat_core()

[]

In [16]:
solver.statistics()

(:added-eqs               29920
 :arith-make-feasible     8
 :arith-max-columns       11
 :binary-propagations     811
 :conflicts               109
 :datatype-accessor-ax    1114
 :datatype-constructor-ax 11816
 :datatype-splits         1944
 :decisions               86083
 :del-clause              42
 :final-checks            8
 :max-generation          4
 :max-memory              30.88
 :memory                  30.88
 :mk-bool-var             4810
 :mk-clause               1138
 :num-allocs              58273624
 :num-checks              1
 :propagations            15765
 :quant-instantiations    1321
 :restarts                3
 :rlimit-count            15307867
 :time                    1.88)

In [17]:
from z3 import Consts, ForAll, Exists, Implies, And, Or

In [17]:
# All VMs have a network interface.
vm, iface = Consts(
    "vm iface", elem_sort
)
vmIfaceAssertion = ForAll(
    [vm],
    Implies(
        elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
        Exists(
            [iface],
            And(
                assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
            )
        )
    )
)
solver.assert_and_track(vmIfaceAssertion, "vm_iface")

In [18]:
# nginx has a source code property
softwareComponent, prop = Consts("softwareComponent prop", elem_sort)
nginxSourceCode = ForAll(
    [softwareComponent],
    Implies(
        And(
            elem_class_f(softwareComponent) == class_["application_SoftwareComponent"],
            attr_rel(softwareComponent, attr["commons_DOMLElement::name"], AData.ss(ss["nginx"]))
        ),
        Exists(
            [prop],
            And(
                elem_class_f(prop) == class_["commons_SProperty"],
                attr_rel(prop, attr["commons_Property::key"], AData.ss(ss["source_code"])),
                assoc_rel(softwareComponent, assoc["commons_DOMLElement::annotations"], prop)
            )
        )
    )
)
solver.assert_and_track(nginxSourceCode, "nginx_source_code")

In [19]:
solver.check()

sat

In [20]:
solver.statistics().memory

66.31

In [21]:
m = solver.model()
print(m)

[attribute_st_types infrastructure_Rule::toPort = True,
 associations sg out_all = True,
 associations i1 http = True,
 associations ag http = True,
 associations vm_flavor con_infra = True,
 associations openstack concrete_vm_image = True,
 attribute_mult_ub commons_BProperty::value = True,
 associations v_img name = True,
 attribute_mult_lb infrastructure_Rule::kind = True,
 associations vm1 name = True,
 attribute_st_types commons_DOMLElement::name = True,
 association_mult_ub concrete_FunctionAsAService::maps = True,
 associations concrete_vm vm_key_name = True,
 associations app openstack = True,
 association_inverse infrastructure_Container::generatedFrom infrastructure_ContainerImage::generatedContainers = True,
 associations app concrete_vm = True,
 associations out_all v_img = True,
 association_mult_ub infrastructure_Container::generatedFrom = True,
 association_st_classes infrastructure_FunctionAsAService::ifaces = True,
 associations net1 ssh = True,
 associations con_infra

In [22]:
from itertools import product
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)